## 均值和方差

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# 解决中文乱码问题（使用SimHei黑体）
plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置中文字体
plt.rcParams['axes.unicode_minus'] = False    # 正常显示负号

from IPython.display import display, HTML

display(HTML('''
<style>
.output_scroll {overflow-y: hidden;}
</style>
'''))



# 设置随机种子保证结果可复现
np.random.seed(42)

In [ ]:
# 生成两组正态分布数据
data1 = np.random.normal(loc=0, scale=1, size=1000)  # 均值0，方差1
data2 = np.random.normal(loc=5, scale=2, size=1000)  # 均值5，方差4

# 可视化直方图
plt.figure(figsize=(8,6))
plt.hist(data1, bins=30, alpha=0.6, label='N(0,1)')
plt.hist(data2, bins=30, alpha=0.6, label='N(5,4)')
plt.legend()
plt.title("两组正态分布数据对比")
plt.xlabel("值")
plt.ylabel("频数")
plt.show()


在正态分布里：

* **均值 (μ)** 决定曲线的中心位置。
* **方差 (σ²)** 决定曲线的“宽度”和“平缓程度”。

具体来说：

* **方差越大**：

  * 数据在均值附近的集中程度降低，曲线更“扁平”，尾部更“厚”，直方图或曲线看起来更“宽”。
* **方差越小**：

  * 数据更集中在均值附近，曲线更“尖锐”，直方图更“窄”。

所以你在图中看到 **N(5,4)** 比 **N(0,1)** 分布更“宽”，就是因为它的标准差 σ = 2，大于前者的 σ = 1。



In [ ]:
# 参数设置
mu = 0
sigma1 = 1   # 方差 = 1
sigma2 = 2   # 方差 = 4
sigma3 = 3   # 方差 = 9

# x 范围
x = np.linspace(-10, 10, 1000)

# 概率密度函数
y1 = stats.norm.pdf(x, mu, sigma1)
y2 = stats.norm.pdf(x, mu, sigma2)
y3 = stats.norm.pdf(x, mu, sigma3)

# 绘制曲线
plt.figure(figsize=(8,6))
plt.plot(x, y1, label="N(0,1)")
plt.plot(x, y2, label="N(0,4)")
plt.plot(x, y3, label="N(0,9)")
plt.title("相同均值，不同方差的正态分布")
plt.xlabel("值")
plt.ylabel("概率密度")
plt.legend()
plt.show()


## 参数估计

In [ ]:
data = np.random.normal(loc=5, scale=2, size=500)

# 2. 参数估计：估计均值和标准差
estimated_mean = np.mean(data)
estimated_std = np.std(data, ddof=1)  # 样本标准差

print(f"估计的均值: {estimated_mean:.2f}")
print(f"估计的标准差: {estimated_std:.2f}")

# 3. 可视化
plt.figure(figsize=(8,5))

# 数据直方图
count, bins, ignored = plt.hist(data, bins=30, density=True, alpha=0.6, color='skyblue', label='数据直方图')

# 拟合的正态分布曲线
x = np.linspace(min(data), max(data), 1000)
pdf = stats.norm.pdf(x, loc=estimated_mean, scale=estimated_std)
plt.plot(x, pdf, 'r-', lw=2, label='拟合正态分布')

plt.title("正态分布参数估计示例")
plt.xlabel("数值")
plt.ylabel("概率密度")
plt.legend()
plt.show()


### 梯度下降最大似然估计

In [ ]:
from IPython.display import display, HTML

# 让 Jupyter 输出区域不再强制滚动
display(HTML('''
<style>
.output_scroll {
    height: auto !important;
    max-height: none !important;
}
</style>
'''))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, expon, poisson

# -----------------------------
# 1. 数据生成
# -----------------------------
np.random.seed(42)
data_normal = np.random.normal(loc=5, scale=2, size=500)
data_expon = np.random.exponential(scale=2, size=500)   # λ=1/2
data_poisson = np.random.poisson(lam=4, size=500)

# -----------------------------
# 2. MLE 梯度上升
# -----------------------------
def mle_normal(data, lr=0.002, iterations=50):
    mu_est, sigma_est = 0.0, 5.0
    mu_hist, sigma_hist = [], []
    for _ in range(iterations):
        grad_mu = np.sum((data - mu_est) / sigma_est**2)
        grad_sigma = np.sum(((data - mu_est)**2 - sigma_est**2) / sigma_est**3)
        mu_est += lr * grad_mu
        sigma_est += lr * grad_sigma
        mu_hist.append(mu_est)
        sigma_hist.append(sigma_est)
    return mu_hist, sigma_hist

def mle_expon(data, lr=0.001, iterations=50):
    lam_est = 0.1
    lam_hist = []
    for _ in range(iterations):
        grad_lam = np.sum(1/lam_est - data)
        lam_est += lr * grad_lam
        lam_hist.append(lam_est)
    return lam_hist

def mle_poisson(data, lr=0.001, iterations=50):
    lam_est = 1.0
    lam_hist = []
    for _ in range(iterations):
        grad_lam = np.sum(data/lam_est - 1)
        lam_est += lr * grad_lam
        lam_hist.append(lam_est)
    return lam_hist

# -----------------------------
# 3. 收敛过程预计算
# -----------------------------
mu_hist, sigma_hist = mle_normal(data_normal)
lam_expon_hist = mle_expon(data_expon)
lam_pois_hist = mle_poisson(data_poisson)

# -----------------------------
# 4. 静态图绘制
# -----------------------------
x = np.linspace(-1, 15, 500)
steps = [9, 19, 49]

fig, axes = plt.subplots(3, 4, figsize=(20, 10))
fig.subplots_adjust(hspace=0.4, wspace=0.3)

# ---- Normal ----
for i, step in enumerate(steps):
    ax = axes[0, i]
    ax.hist(data_normal, bins=30, density=True, alpha=0.3, color='skyblue', label="数据")
    ax.plot(x, norm.pdf(x, 5, 2), "g--", lw=2, label="真实分布")
    mu, sigma = mu_hist[step], sigma_hist[step]
    ax.plot(x, norm.pdf(x, mu, sigma), "b-", lw=2, label="估计分布")
    ax.set_title(f"Normal step={step+1}\nμ={mu:.2f}, σ={sigma:.2f}")
    ax.legend()

# Normal 收敛曲线
ax = axes[0, 3]
ax.plot(mu_hist, label="μ 估计")
ax.axhline(5, color='g', linestyle="--", label="μ 真值")
ax.plot(sigma_hist, label="σ 估计")
ax.axhline(2, color='r', linestyle="--", label="σ 真值")
ax.set_title("Normal 参数收敛")
ax.legend()

# ---- Exponential ----
x_exp = np.linspace(0, 10, 500)
for i, step in enumerate(steps):
    ax = axes[1, i]
    ax.hist(data_expon, bins=30, density=True, alpha=0.3, color='skyblue', label="数据")
    ax.plot(x_exp, expon.pdf(x_exp, scale=2), "g--", lw=2, label="真实分布")
    lam = lam_expon_hist[step]
    ax.plot(x_exp, expon.pdf(x_exp, scale=1/lam), "b-", lw=2, label="估计分布")
    ax.set_title(f"Exponential step={step+1}\nλ={lam:.2f}")
    ax.legend()

# Exponential 收敛曲线
ax = axes[1, 3]
ax.plot(lam_expon_hist, label="λ 估计")
ax.axhline(0.5, color='g', linestyle="--", label="λ 真值")
ax.set_title("Exponential 参数收敛")
ax.legend()

# ---- Poisson ----
vals, counts = np.unique(data_poisson, return_counts=True)
for i, step in enumerate(steps):
    ax = axes[2, i]
    ax.bar(vals, counts/len(data_poisson), alpha=0.3, color='skyblue', label="数据")
    ax.plot(vals, poisson.pmf(vals, 4), "g--", lw=2, label="真实分布")
    lam = lam_pois_hist[step]
    ax.plot(vals, poisson.pmf(vals, lam), "b-", lw=2, label="估计分布")
    ax.set_title(f"Poisson step={step+1}\nλ={lam:.2f}")
    ax.legend()

# Poisson 收敛曲线
ax = axes[2, 3]
ax.plot(lam_pois_hist, label="λ 估计")
ax.axhline(4, color='g', linestyle="--", label="λ 真值")
ax.set_title("Poisson 参数收敛")
ax.legend()

plt.show()


发现同样的学习率，正态分布收敛的很慢，指数分布收敛最快，但后面一直震荡，泊松分布式是刚开始收敛的很快，到最后几乎没有梯度了，不收敛了。

你观察到的三种分布 MLE 收敛特性完全合理，这其实跟**梯度公式的尺度、参数初始化以及学习率选择**息息相关。我帮你分析一下原因，并给出可行的改进思路：

---

### 1️⃣ 正态分布收敛慢

你的梯度公式是：

$$
\frac{\partial \ell}{\partial \mu} = \sum_i \frac{x_i - \mu}{\sigma^2}, \quad
\frac{\partial \ell}{\partial \sigma} = \sum_i \frac{(x_i-\mu)^2 - \sigma^2}{\sigma^3}
$$

* **问题原因**：

  * 梯度被 `1/σ²` 和 `1/σ³` 缩放，如果 σ 很大或很小，梯度会很小或很大。
  * 初始化 σ=5，略偏大，导致初期 μ 梯度较小，更新慢。
  * 学习率 lr=0.001 太小，所以更新慢。
* **改进方法**：

  * 调整学习率，例如 lr=0.01\~0.05。
  * 对 σ 使用对数参数化：σ = exp(s)，优化 s 而不是 σ，这样梯度更稳定。
  * 对 μ、σ 梯度做标准化或归一化。

---

### 2️⃣ 指数分布收敛快但震荡

梯度公式：

$$
\frac{\partial \ell}{\partial \lambda} = \sum_i \frac{1}{\lambda} - \sum_i x_i
$$

* **问题原因**：

  * 梯度与 λ 的倒数成正比，如果 λ 很小（初始化 0.1），梯度非常大。
  * lr 太大 + 梯度大 → 快速收敛但容易 overshoot → 后期震荡。
* **改进方法**：

  * 使用更合理初始化 λ ≈ 1/mean(data)，这样初始梯度不会太大。
  * 使用 **自适应学习率** 或 **梯度衰减**。
  * 可以用 **analytical MLE** λ = 1 / mean(data)，指数分布 MLE 本身有闭式解，所以梯度下降不一定是最优选择。

---

### 3️⃣ 泊松分布收敛快到最后停滞

梯度公式：

$$
\frac{\partial \ell}{\partial \lambda} = \sum_i \frac{x_i}{\lambda} - n
$$

* **问题原因**：

  * 梯度中有 1/λ，当 λ 接近真实值时梯度很小。
  * 学习率 lr 太小，导致后期几乎不再更新 → “停滞”。
* **改进方法**：

  * 增大学习率或者使用 **自适应优化器**（如 Adam）。
  * 可以尝试 **牛顿法**，泊松分布 MLE 有闭式解 λ = mean(data)，收敛非常快。

---

### 🔑 总结规律

| 分布 | 梯度特点         | 初始收敛 | 后期问题 | 改进思路                    |
| -- | ------------ | ---- | ---- | ----------------------- |
| 正态 | 梯度被 σ²/σ³ 缩放 | 慢    | 很慢收敛 | 提大学习率，对 σ 用对数参数化，梯度归一化  |
| 指数 | 梯度 ∝ 1/λ     | 快    | 震荡   | 合理初始化 λ，降低 lr 或使用自适应 lr |
| 泊松 | 梯度 ∝ 1/λ     | 快    | 后期停滞 | 自适应 lr 或牛顿法，直接用 MLE 闭式解 |

---

## 假设检验
### 单样本 t 检验

In [ ]:
data = np.random.normal(loc=5.2, scale=1.5, size=30)  # 实际均值略大于5

# 2. 单样本 t 检验
# H0: 均值 = 5
t_stat, p_value = stats.ttest_1samp(data, popmean=5)

print(f"t统计量: {t_stat:.3f}")
print(f"p值: {p_value:.3f}")

# 3. 可视化数据分布与均值
plt.figure(figsize=(8,5))

# 数据直方图
plt.hist(data, bins=10, alpha=0.6, color='skyblue', density=True, label='样本数据')

# 样本均值和假设均值
sample_mean = np.mean(data)
plt.axvline(sample_mean, color='red', linestyle='--', label=f'样本均值={sample_mean:.2f}')
plt.axvline(5, color='green', linestyle='--', label='假设均值=5')

# 标准正态曲线（参考）
x = np.linspace(min(data)-1, max(data)+1, 100)
pdf = stats.norm.pdf(x, loc=sample_mean, scale=np.std(data, ddof=1))
plt.plot(x, pdf, 'r-', lw=2, alpha=0.6)

plt.title("单样本 t 检验示例")
plt.xlabel("数值")
plt.ylabel("概率密度")
plt.legend()
plt.show()


### 1. t 统计量解释

t 统计量公式为：

$$
t = \frac{\bar{x} - \mu_0}{s / \sqrt{n}}
$$

* $\bar{x}$ = 样本均值
* $\mu_0$ = 假设均值
* $s$ = 样本标准差
* $n$ = 样本大小

你得到 **t = -0.787**，表示样本均值略低于假设均值，但偏离不大（负号仅表示方向）。

---

### 2. p 值解释

* **p = 0.438**（双尾检验）
* 解释：在原假设成立的情况下，观察到像你样本这样偏离假设均值的情况，概率约 **43.8%**。
* **常用显著性水平 α = 0.05**
* 因为 **p > 0.05**，无法拒绝原假设。

**结论**：你的数据与假设均值 5 没有显著差异。

---

### 3. 可视化解释

直观上：

* 样本均值略低于 5（红色虚线偏左）
* 绿色虚线表示假设均值 5
* 数据分布的波动范围大于红色虚线与绿色虚线的差距
* 所以 t 统计量小，p 值大 → 没有显著差异
